Refer paper Language Models are Unsupervised Multitask Learners
https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf


GPT-2[124M] has 12 layers and $d_{model}$ = 768

It is decoder only model.

Variables are named to follow the schema of Hugging Face Transformers code.  
The following variables should be exactly the same:

`transformer.wte.weight`  
`transformer.wpe.weight`  
`transformer.h.0.attn.c_attn.weight`  
`transformer.h.0.attn.c_proj.weight`  
`transformer.h.0.mlp.c_fc.weight`  
`transformer.h.0.mlp.c_proj.weight`  
`transformer.ln_f.weight`  
`lm_head.weight`


In [ ]:
from transformers import GPT2LMHeadModel
from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import tiktoken
import math

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model_gpt2 = GPT2LMHeadModel.from_pretrained("gpt2") #124M
sd = model_gpt2.state_dict()
for k,v in sd.items():
  print(k, v.shape)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model = 'gpt2')
generator("Hello, I'm a language model,", max_new_tokens=30, num_return_sequences=5)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, and I'm a man. I'm a man. I'm a man. I'm a man. I'm a man. I'm a man"},
 {'generated_text': "Hello, I'm a language model, and I know that it's not easy to do the right thing.\n\nBut I want to share with you two things.\n\nThe first"},
 {'generated_text': "Hello, I'm a language model, I'm a designer.\n\nAnd I'm a programmer too.\n\nMy first book, The Language of Programming, was written in the late"},
 {'generated_text': "Hello, I'm a language model, and I'm pretty sure I understand this.\n\nAnd, as I said, I'm a language model, and I'm pretty sure I understand"},
 {'generated_text': "Hello, I'm a language model, not a language.\n\nThe question is not why I'm different from a language model, but why I'm different from a language model.\n"}]

#GPT-2

In [ ]:

class CausalSelfAttention(nn.Module):
  def __init__(self, config):
    super().__init__()
    # make sure hidden dim is a multiple of no. of heads
    assert config.n_embed % config.n_head == 0

    # a single linear layer to compute Q, K, V simultaneously
    self.c_attn=nn.Linear(config.n_embed, 3 * config.n_embed)

    # output projection
    self.c_proj = nn.Linear(config.n_embed, config.n_embed)
    self.c_proj.NANOGPT_SCALE_INIT = 1 # flag for weight initialization of c_proj, use std = 0.02/sqroot(num layers)

    # regularization
    self.n_head = config.n_head
    self.n_embed = config.n_embed

    # not really a bias, more of a mask, but following OpenAI naming convention
    self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                         .view(1, 1,config.block_size, config.block_size ))



  def forward(self, x):
    B, T, C = x.size()  # Batch size, sequence length, n_embed
    qkv= self.c_attn(x)
    q,k,v = qkv.split(self.n_embed, dim=2)
    k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

    # explanation : C = n_head * head_size
    # k.shape = (B, T, n_head, head_size)
    # k = k.transpose(1, 2)
    # Before transpose: (B, T, n_head, head_size)
    # After transpose:  (B, n_head, T, head_size)

    # similar for q and v
    q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
    v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

    # Attention
    # att = (q @ k.transpose(-2,-1)) * (1.0 / math.sqrt(k.size[-1]))
    # att = att.masked_fill(self.bias[:, :, :T, :T] == 0.0, float('-inf'))
    # att = F.softmax(att, dim=-1)
    # y = att @ v # (B, nh, T, hs) x (B, nh, T, hs) -> (B, nh, T, hs), basically a weighted sum of values

    y = F.scaled_dot_product_attention(q, k, v, is_causal=True) # flash attention

    y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

    # output projection
    y = self.c_proj(y)

    return y





In [ ]:
class MLP(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.c_fc = nn.Linear(config.n_embed, 4 * config.n_embed)         # ffn. increasing hidden dim size increases capacity of model to learn, 4*embed dim is just design choice
    self.gelu = nn.GELU(approximate='tanh')                            # activation
    self.c_proj = nn.Linear( 4 * config.n_embed, config.n_embed) # projection

  def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

In [ ]:
class Block(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.ln_1 = nn.LayerNorm(config.n_embed)  # layer norm 1
    self.attn = CausalSelfAttention(config) # causal attention
    self.ln_2 = nn.LayerNorm(config.n_embed) # layer norm 2
    self.mlp = MLP(config) # fnn

  def forward(self, x):
    x = x + self.attn(self.ln_1(x))
    x = x + self.mlp(self.ln_2(x))
    return x

In [ ]:
@dataclass
class GPTConfig:
  block_size : int = 1024    # max sequence length
  vocab_size : int = 50257   # number of tokens: 50,000 BPE merges + 256 bytes tokens + 1 <|endoftext|> token
  n_layer : int = 12
  n_head : int = 12
  n_embed : int = 768

class GPT(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.config=config

    self.transformer=nn.ModuleDict(dict(
        wte = nn.Embedding(config.vocab_size, config.n_embed),  # weights for token embeddings
        wpe = nn.Embedding(config.block_size, config.n_embed),  # weights for positional embeddings
        h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]), # block for each layer
        ln_f = nn.LayerNorm(config.n_embed),  # final layer normalisation
        ))
    self.lm_head = nn.Linear(config.n_embed, config.vocab_size,bias=False) # last second, linear layer

    # weight-sharing scheme
    self.transformer.wte.weight = self.lm_head.weight

    # initialize parameters
    self.apply(self._init_weights)

  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      std = 0.02
      if hasattr(module, 'NANOGPT_SCALE_INIT'): # will be true only for output projection, `c_proj` layer
        std *= (2 * self.config.n_layer) ** -0.05 # scale std by 1/sqrt(no_of_layers) acc to GPT paper
        # we are doing 2 * no of layers bcoz every layer has 2 blocks that add to residual stream - attention and then mlp
        torch.nn.init.normal_(module.weight, mean=0.0, std = std) # inititalise weights according to gpt2 official code, i.e., mean 0, std 0.02 for weights
      # sqroot n thing is done to control the growth of activations in residual stream in forward pass as each residual stream adds its data so we scale down every contribution to residual stream
        torch.nn.init.zeros_(module.bias) # and normal initialisation for bias
    elif isinstance(module, nn.Embedding):
      torch.nn.init.normal_(module.weight, mean=0.0, std = 0.02)



  def forward(self, idx, targets=None):
    # idx (B, T) Batch size, B sequences, each of length T stacked up, T<=block_size
    B, T = idx.size()
    assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is only {self.config.block_size}"
    # forward the token and posisition embeddings
    pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T), arange iterate from 0 to T
    pos_emb = self.transformer.wpe(pos) # shape (T, n_embd) # identical for every single row (batch)
    tok_emb = self.transformer.wte(idx) # shape (B, T, n_embd)
    x = tok_emb + pos_emb # internal broadcasting
    # forward the blocks of transformer
    for block in self.transformer.h:
      x = block(x)
    # forward the final layernorm
    x = self.transformer.ln_f(x)
    # forward the final classifier
    logits=self.lm_head(x) # (B, T, vocab_size)
    loss=None
    if targets is not None:
      loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1)) # logits - (B*T, vocab_size)
    return logits, loss


  @classmethod
  def from_pretrained(cls, model_type):
      # Loads pretrained GPT-2 model weights from huggingface
      assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
      print("loading weights from pretrained gpt: %s" % model_type)

      # n_layer, n_head and n_embed are determined from model_type
      config_args = {
          'gpt2':         dict(n_layer=12, n_head=12, n_embed=768),  # 124M params
          'gpt2-medium':  dict(n_layer=24, n_head=16, n_embed=1024), # 350M params
          'gpt2-large':   dict(n_layer=36, n_head=20, n_embed=1280), # 774M params
          'gpt2-xl':      dict(n_layer=48, n_head=25, n_embed=1600), # 1558M params
      }[model_type]
      config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
      config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
      # create a from-scratch initialized minGPT model
      config = GPTConfig(**config_args)
      model = GPT(config)
      sd = model.state_dict()
      sd_keys = sd.keys()
      sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

      # init a huggingface/transformers model
      model_hf = GPT2LMHeadModel.from_pretrained(model_type)
      sd_hf = model_hf.state_dict()

      # copy while ensuring all of the parameters are aligned and match in names and shapes
      sd_keys_hf = sd_hf.keys()
      sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
      sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
      transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
      # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
      # this means that we have to transpose these weights when we import them
      assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
      for k in sd_keys_hf:
          if any(k.endswith(w) for w in transposed):
              # special treatment for the Conv1D weights we need to transpose
              assert sd_hf[k].shape[::-1] == sd[k].shape
              with torch.no_grad():
                  sd[k].copy_(sd_hf[k].t())
          else:
              # vanilla copy over the other parameters
              assert sd_hf[k].shape == sd[k].shape
              with torch.no_grad():
                  sd[k].copy_(sd_hf[k])

      return model

  def configure_optimizers(self, weight_decay, learning_rate, device_type):
      # start with all of the candidate parameters (that require grad)
      param_dict = {pn: p for pn, p in self.named_parameters()}
      param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
      # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
      # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
      decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
      nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
      optim_groups = [
          {'params': decay_params, 'weight_decay': weight_decay},
          {'params': nodecay_params, 'weight_decay': 0.0}
      ]
      num_decay_params = sum(p.numel() for p in decay_params)
      num_nodecay_params = sum(p.numel() for p in nodecay_params)
      if master_process:
          print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
          print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
      # Create AdamW optimizer and use the fused version if it is available
      fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
      use_fused = fused_available and device_type == "cuda"
      if master_process:
          print(f"using fused AdamW: {use_fused}")
      optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused)
      return optimizer



In [ ]:
# model=GPT.from_pretrained("gpt2")
model = GPT(GPTConfig()) # random initialization

In [ ]:
model.eval() #put model into eval mode when not training anything and just using the model
model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
num_return_sequences=5
max_length=30

In [ ]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model,") # (8, )
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) # (5, 8)
x = tokens.to('cuda')

In [ ]:
tokens.shape

torch.Size([5, 8])

##Generate before Training

In [ ]:
# generate.  right now x is (B, T) where B=5 and T=8
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length: # add a column of new indices, i.e. add new token for each of the 5 sequences
    # forward the model to get the logits
    with torch.no_grad():
        logits, _ = model(x) # (B, T, vocab_size)

        # get the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)

        # get the probabilities
        probs = F.softmax(logits, dim=-1)

        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1) # ensures no sampling of very rare tokens

        # select a token from the top-k probabilities
        ix = torch.multinomial(topk_probs, 1) # (B, 1)

        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)

        # append to the sequence
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for i in range(num_return_sequences):
  tokens=x[i, :max_length].tolist()
  decoded=enc.decode(tokens)
  print(">", decoded)

> Hello, I'm a language model, PTS headset rivaldealDN carried Furthermore headset universe makesدdatDNangs192 strained Tough depositedierrez convoluted Designs
> Hello, I'm a language model, criticisms Cullen reviewer universe headsetGet modifications herdpowers Sul Bj immigrant   repeatedly − Pop masse masse − quantGet brew
> Hello, I'm a language model, accessory unbeliev unbelievshaped accessory carried"), reducediefrizBradesternIsariz Shy recommendremote representing ger improperlyMagazine Turtle
> Hello, I'm a language model,Ham criticisms aggressively rivalDN islandsDN Dragonbound Russo ad encompasses ost RFCAdding some comboAyylum Travels beams comprehensiveserial
> Hello, I'm a language model,Bradannie697ryptBrad summerBornEat taxp willingly summerophone some protestsanc improperly Cullen Mar79 someKn reiter


##Data Processing

We want to feed the token sequences to a transformer

In [ ]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
with open("input.txt", 'r') as f:
  text = f.read()
data = text[:1000]
print(data)
tokens = enc.encode(data) # encode data
B, T = 4, 32
buf = torch.tensor(tokens[:B*T+1]) # take only B*T tokens to manage size and an additional token which will be used in output y as target to nth token
buf = buf.to(device) # it doesn't just move the data to gpu, it creates a new memory on gpu
x = buf[:-1].view(B, T) # all tokens except last
y = buf[1:].view(B, T) # targets will be from 1st token
print(x)
print(y)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



Its like for 0th token, target is 1st token, for 0 and 1st token, target is 2nd token and so on (masked prediction) and so our x goes from 0 to B*T and y goes from 1 to B*T+1

In [ ]:
model.to(device)
logits, loss = model(x, y)
logits.shape

torch.Size([4, 32, 50257])

In [ ]:
print(loss)

tensor(11.0363, device='cuda:0', grad_fn=<NllLossBackward0>)


At inititalization, we expect loss to be roughly around -ln(1/vocab_size)[NLL]1
 = 10.8 here,  since at initialization, probability of any word is same i.e., 1/50257.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
for i in range(50):
  optimizer.zero_grad()
  logits, loss = model(x,y)
  loss.backward()
  optimizer.step()
  print(f"step : {i}, loss : {loss.item()}")
# Here we overfit a single batch, now lets move to training on all batches!

step : 0, loss : 11.036296844482422
step : 1, loss : 8.401263236999512
step : 2, loss : 7.556160926818848
step : 3, loss : 7.823549270629883
step : 4, loss : 6.896722793579102
step : 5, loss : 6.51249361038208
step : 6, loss : 6.098946571350098
step : 7, loss : 5.710570335388184
step : 8, loss : 5.27562141418457
step : 9, loss : 4.860488414764404
step : 10, loss : 4.485625267028809
step : 11, loss : 4.047159194946289
step : 12, loss : 3.5754752159118652
step : 13, loss : 3.153848171234131
step : 14, loss : 2.7754809856414795
step : 15, loss : 2.3487753868103027
step : 16, loss : 2.044778347015381
step : 17, loss : 1.7443827390670776
step : 18, loss : 1.439927577972412
step : 19, loss : 1.1365606784820557
step : 20, loss : 0.8545939922332764
step : 21, loss : 0.6557171940803528
step : 22, loss : 0.49248164892196655
step : 23, loss : 0.35574260354042053
step : 24, loss : 0.25000423192977905
step : 25, loss : 0.18138493597507477
step : 26, loss : 0.13208100199699402
step : 27, loss : 0.09

In [ ]:
class DataLoaderLite:
  def __init__(self, B, T):
    self.B=B
    self.T=T

    # at init, load tokens from disk and store them in memory
    with open('input.txt', 'r') as f:
      text = f.read()
    enc = tiktoken.get_encoding('gpt2')
    tokens = enc.encode(text)
    self.tokens = torch.tensor(tokens)
    print(f"Loaded {len(self.tokens)} tokens")
    print(f"1 epoch = {len(self.tokens) // (B*T)} batches")
    self.current_size = 0 # state

  def next_batch(self):
    B, T = self.B, self.T
    buf = self.tokens[self.current_size:self.current_size + B*T + 1] # +1 coz we need it in 'y'
    # buf = buf.to(device) dont do this here to save space on gpu
    x = buf[:-1].view(B, T) # inputs
    y = buf[1:].view(B, T) # targets
    self.current_size += B*T # advance position in tensor
    # if loading next batch would be out of bounds, reset
    if self.current_size + B*T + 1 > len(self.tokens):
      self.current_size = 0
    return x, y

In [ ]:
for name, m in model.named_modules():
    if hasattr(m, "bias") and m.bias is None:
        print("no-bias module:", name, type(m))


no-bias module: lm_head <class 'torch.nn.modules.linear.Linear'>


In [ ]:
train_loader = DataLoaderLite(B=4, T=32)
model = GPT(GPTConfig())
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
for i in range(50):
  x, y = train_loader.next_batch()
  x, y = x.to(device), y.to(device)
  optimizer.zero_grad()
  with torch.autocast(device_type=device, dtype=torch.bfloat16):
    logits, loss = model(x, y)
  loss.backward()
  optimizer.step()
  print(f"step : {i}, loss : {loss.item()}")

Loaded 338025 tokens
1 epoch = 2640 batches
step : 0, loss : 10.81787109375
step : 1, loss : 9.78515625
step : 2, loss : 9.002197265625
step : 3, loss : 9.053466796875
step : 4, loss : 8.6226806640625
step : 5, loss : 8.3692626953125
step : 6, loss : 9.005859375
step : 7, loss : 8.7735595703125
step : 8, loss : 8.06494140625
step : 9, loss : 7.9627685546875
step : 10, loss : 8.40625
step : 11, loss : 7.395263671875
step : 12, loss : 7.79168701171875
step : 13, loss : 7.4029541015625
step : 14, loss : 7.56097412109375
step : 15, loss : 7.36614990234375
step : 16, loss : 7.43896484375
step : 17, loss : 8.29638671875
step : 18, loss : 7.3306884765625
step : 19, loss : 7.87353515625
step : 20, loss : 7.483642578125
step : 21, loss : 7.7362060546875
step : 22, loss : 6.419921875
step : 23, loss : 6.866455078125
step : 24, loss : 6.86273193359375
step : 25, loss : 6.73077392578125
step : 26, loss : 6.82000732421875
step : 27, loss : 7.6192626953125
step : 28, loss : 7.19854736328125
step : 2

# Optimization